## Single Step Tool Use
- Tool use can be executed in two modes, single step and multi step
- In single step tool use an LLM calls one or many tools in parallel. The tools then return the results, which the LLM can use to generate its response.
- In this notebook we will go through how to use multiple tools, when not to use tools and how to use tools in a chat setting for maintaining context.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv('COHERE_API_KEY', '')

In [3]:
# Run this cell if you want to make your display wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
import json
import cohere

c:\Users\heena\Desktop\Genetative_AI\genai_env\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [5]:
co = cohere.Client(api_key=cohere_api_key)

### Step 1. Create Tools
- Here we will create 2 tools, one will be a dummy sales database and the other tool will be a product catalog.

In [6]:
def daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    # Mock database containing daily sales reports
    sales_database = {
    '2023-09-28': {'total_sales_amount': 5000,'total_units_sold': 100},
    '2023-09-29': {'total_sales_amount': 10000,'total_units_sold': 250},
    '2023-09-30': {'total_sales_amount': 8000,'total_units_sold': 200}
    }
    
    report = sales_database.get(day, {})
    
    if report:
        return {
            'date': day,
            'summary': f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}"
        }
    else:
        return {'date': day, 'summary': 'No sales data available for this day.'}

In [7]:
def product_database(category: str) -> dict:
    """
    Function to retrieve products for the given category
    """
    
    # Mock product catalog
    product_catalog = {
        'Electronics': [
            {'product_id': 'E1001', 'name': 'Smartphone', 'price': 500, 'stock_level': 20},
            {'product_id': 'E1002', 'name': 'Laptop', 'price': 1000, 'stock_level': 15},
            {'product_id': 'E1003', 'name': 'Tablet', 'price': 300, 'stock_level': 25},
        ],
        'Clothing': [
            {'product_id': 'C1001', 'name': 'T-Shirt', 'price': 20, 'stock_level': 100},
            {'product_id': 'C1002', 'name': 'Jeans', 'price': 50, 'stock_level': 80},
            {'product_id': 'C1003', 'name': 'Jacket', 'price': 100, 'stock_level': 40},
        ]
    }
    
    products = product_catalog.get(category, [])
    return {
        'category': category,
        'products': products
    }

In [8]:
functions_map = {
    "daily_sales_report": daily_sales_report,
    "product_database": product_database
}

### Step 2. Define Tool Schema
- Define schema for both the tools, to explain the LLMs what the tools do and the parameters they take.

In [9]:
tools = [
    {
        "name": "daily_sales_report",
        "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
                "type": "str",
                "required": True
            }
        }
    },
    {
        "name": "product_database",
        "description": "A database that contains information about all the products of this company, including categories, prices, and stock levels.",
        "parameter_definitions": {
            "category": {
                "description": "Retrieves product information data for all products in this category.",
                "type": "str",
                "required": True
            }
        }
    }
]

### Step 3 Create Custom Preamble (Optional)

In [10]:
preamble = """## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

### Single Step Tool Use
- Below is the workflow for single step tool use
    1. Get the user message.
    2. Call the Chat endpoint for tool call generation.
    3. If the response contains at least one tool call, execute the tool call(s) and get the tool results.
    4. Generate the final response with citations.

- To make this a chat application, we will keep steps 1, 2 and 3 in a loop so we can handle multiple tool calls.
- We will also send chat history so we can handle multi-turn chat scenarios.

In [11]:
model = "command-r-plus"

def run_assistant(message, chat_history=[]):
    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("="*50)

    # Step 2: Generate tool calls (if any)    
    response = co.chat(
        message=message,
        model=model,
        preamble=preamble,
        tools=tools,
        chat_history=chat_history,
        force_single_step=True
    )

    while response.tool_calls:
        tool_calls = response.tool_calls
        
        if response.text:
            print("Tool plan:")
            print(response.text,"\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("="*50)
        
        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            tool_call = {"name": tc.name, "parameters": tc.parameters}
            tool_output = functions_map[tc.name](**tc.parameters)
            tool_results.append({"call": tool_call, "outputs": [tool_output]})
        
        # Step 4: Generate response and citations                
        response = co.chat(
            message="",
            model=model,
            preamble=preamble,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history
        )

        # Update the chat history
        chat_history = response.chat_history
        
    # Print final response
    print("Final response:")
    print(response.text)
    print("="*50)
    
    # Print citations (if any)
    if response.citations:
        print("Citations:")
        for citation in response.citations:
            print(citation)
        print("\nCited Documents:")
        for document in response.documents:
            print(document)
        print("="*50)
    
    return chat_history

In [12]:
chat_history = run_assistant("Can you provide a sales summary for 29th September 2023?")

Question:
Can you provide a sales summary for 29th September 2023?
Tool calls:
Tool name: daily_sales_report | Parameters: {'day': '2023-09-29'}
Final response:
On 29 September 2023, we sold 250 units and made a total of $10,000.
Citations:
start=30 end=39 text='250 units' document_ids=['daily_sales_report:0:2:0']
start=51 end=67 text='total of $10,000' document_ids=['daily_sales_report:0:2:0']

Cited Documents:
{'date': '2023-09-29', 'id': 'daily_sales_report:0:2:0', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250', 'tool_name': 'daily_sales_report'}


### Single-Step, Parallel Tool Use
- The model can decide that more than one tool is required to provide a response. 
- This means calling multiple tools in parallel within the same step. This can either be:
    - Calling different tools in parallel.
    - Calling the same tool multiple times in parallel.
    - or a combination of both.

In [13]:
# Here we will ask about sales summary of two different dates and stock level information of a product category
chat_history = run_assistant("Can you provide a sales summary for 28th and 29th September 2023 as well as the stock level of the products in the 'Electronics' category?")

Question:
Can you provide a sales summary for 28th and 29th September 2023 as well as the stock level of the products in the 'Electronics' category?
Tool calls:
Tool name: daily_sales_report | Parameters: {'day': '2023-09-28'}
Tool name: daily_sales_report | Parameters: {'day': '2023-09-29'}
Tool name: product_database | Parameters: {'category': 'Electronics'}
Final response:
On 28 September 2023, the total sales amount was 5000, with 100 units sold. The following day, 29 September 2023, the total sales amount was 10000, with 250 units sold. 

Here is the stock level of the products in the 'Electronics' category:
- Smartphone (E1001) - 20
- Laptop (E1002) - 15
- Tablet (E1003) - 25
Citations:
start=3 end=20 text='28 September 2023' document_ids=['daily_sales_report:0:2:0']
start=26 end=53 text='total sales amount was 5000' document_ids=['daily_sales_report:0:2:0']
start=60 end=74 text='100 units sold' document_ids=['daily_sales_report:0:2:0']
start=95 end=112 text='29 September 2023' d

### Directly Answering
- A key attribute of tool use systems is to choose the right tools for a task.
- This includes the ability not to use a tool and instead respond to user message directly.
- This could happen if the user's question can be answered directly or none of the tools are helpful to answer the question.

In [18]:
# 1. The model can directly answer the user's question
chat_history = run_assistant("Give me 3 concise tips on how to build a good company")

Question:
Give me 3 concise tips on how to build a good company
Final response:
1. Have a clear vision and strategy.
2. Hire the right people and treat them well.
3. Focus on delivering value to your customers.


In [19]:
# 2. The model does not possess the answer to the question and there are no available tools as well.
chat_history = run_assistant("How many employees does this company have?")

Question:
How many employees does this company have?
Tool calls:
Tool name: product_database | Parameters: {'category': 'employees'}
Final response:
I'm sorry, I don't have access to information about the number of employees at this company.
